Kolekcia všetkých importov použitých v tomto projekte

In [1]:
import pandas as pd
import numpy as np
import math

Načítaj csv súbor, v ktorom už sú len riadky v danom rozsahu

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Škola/WM/semestralny_projekt/wm2020.csv', ',')
data

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent
0,35.194.1.1,-,-,19/Nov/2017:06:27:21 +0100,GET /en/media-a-marketing/foto/image?format=ra...,200,494442,-,ltx71 - (http://ltx71.com/)
1,34.224.83.229,-,-,19/Nov/2017:06:27:24 +0100,GET /media-a-marketing/foto/krst-atlasu?format...,200,4540,https://www.google.com/,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...
2,35.184.189.105,-,-,19/Nov/2017:06:27:31 +0100,GET /images/univerzita/Uradna_tabula/VS_FPV_20...,200,515664,-,ltx71 - (http://ltx71.com/)
3,35.194.1.1,-,-,19/Nov/2017:06:27:33 +0100,GET /media-a-marketing/foto/image?format=raw&i...,200,80893,-,ltx71 - (http://ltx71.com/)
4,185.92.73.31,-,-,19/Nov/2017:06:28:01 +0100,GET /administrator/index.php HTTP/1.1,200,6429,https://www.ukf.sk/administrator/index.php,Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_55_...
...,...,...,...,...,...,...,...,...,...
1305687,85.25.210.41,-,-,19/Nov/2017:06:26:45 +0100,GET /robots.txt HTTP/1.1,200,4230,-,Mozilla/5.0 (compatible; seoscanners.net/1; +s...
1305688,35.184.189.105,-,-,19/Nov/2017:06:26:44 +0100,GET /images/univerzita/Uradna_tabula/VS_FPV_20...,200,511440,-,ltx71 - (http://ltx71.com/)
1305689,62.138.0.25,-,-,19/Nov/2017:06:26:46 +0100,GET /robots.txt HTTP/1.1,200,4230,-,Mozilla/5.0 (compatible; seoscanners.net/1; +s...
1305690,157.55.39.148,-,-,19/Nov/2017:06:27:04 +0100,GET /media-a-marketing/34-studium/studensky-zi...,200,12209,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...


###1. očistite dátový súbor od nepotrebných údajov *(RequestMethod/Version, StatusCode, URL)*


In [3]:
request = ['HEAD', 'POST']
suffix = ['.bmp','.jpg','.jpeg','.png','.gif','.JPG','.css','.flv','.ico','.swf','.rss','.xml','.cur','.js','.json','.svg','.svg','.woff','.eot']

Nechaj v dataframe len riadky, ktoré spĺňajú požiadavky

In [4]:
data = data[~data.url.str.contains('|'.join(suffix))]
data = data[~data.req_code.astype(str).str.match(r'1* | 4* | 5* ')]
data = data[~data.url.str.contains('|'.join(request))]
data = data[~data.url.str.contains('GET /navbar/')]
data = data[~data.url.str.contains('ctrl=cron')]
data

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent
0,35.194.1.1,-,-,19/Nov/2017:06:27:21 +0100,GET /en/media-a-marketing/foto/image?format=ra...,200,494442,-,ltx71 - (http://ltx71.com/)
2,35.184.189.105,-,-,19/Nov/2017:06:27:31 +0100,GET /images/univerzita/Uradna_tabula/VS_FPV_20...,200,515664,-,ltx71 - (http://ltx71.com/)
3,35.194.1.1,-,-,19/Nov/2017:06:27:33 +0100,GET /media-a-marketing/foto/image?format=raw&i...,200,80893,-,ltx71 - (http://ltx71.com/)
4,185.92.73.31,-,-,19/Nov/2017:06:28:01 +0100,GET /administrator/index.php HTTP/1.1,200,6429,https://www.ukf.sk/administrator/index.php,Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_55_...
6,194.1.157.190,-,-,19/Nov/2017:06:28:06 +0100,GET / HTTP/1.1,200,16195,-,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
...,...,...,...,...,...,...,...,...,...
1305686,35.188.119.38,-,-,19/Nov/2017:06:26:42 +0100,GET /en/student-services/support-for-disabled-...,200,12662,-,ltx71 - (http://ltx71.com/)
1305687,85.25.210.41,-,-,19/Nov/2017:06:26:45 +0100,GET /robots.txt HTTP/1.1,200,4230,-,Mozilla/5.0 (compatible; seoscanners.net/1; +s...
1305688,35.184.189.105,-,-,19/Nov/2017:06:26:44 +0100,GET /images/univerzita/Uradna_tabula/VS_FPV_20...,200,511440,-,ltx71 - (http://ltx71.com/)
1305689,62.138.0.25,-,-,19/Nov/2017:06:26:46 +0100,GET /robots.txt HTTP/1.1,200,4230,-,Mozilla/5.0 (compatible; seoscanners.net/1; +s...


###2. očistite dátový súbor od prístupov robotov vyhľadávacích služieb
* identifikujte robotov na základe prístupu k súboru robots.txt (URL)
* identifikujte robotov na základe poľa User-Agent (Agent)


Získaj ip robotov do pola

In [5]:
ipArray = data[data.url.str.contains('robots.txt')]['clientIp'].unique()
print("Size: " + str(ipArray.size))

Size: 257


Odstráň ip robotov z csv súboru

In [6]:
data = data[~data.clientIp.str.contains('|'.join(ipArray))]

Odstráň robotov vďaka kľučovým slovám ako *(bot, crawl, spider)*

In [7]:
robotArray = ['bot', 'crawl', 'spider']
data = data[~data.useragent.str.contains('|'.join(robotArray), na=False)]
data

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent
4,185.92.73.31,-,-,19/Nov/2017:06:28:01 +0100,GET /administrator/index.php HTTP/1.1,200,6429,https://www.ukf.sk/administrator/index.php,Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_55_...
6,194.1.157.190,-,-,19/Nov/2017:06:28:06 +0100,GET / HTTP/1.1,200,16195,-,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
73,139.162.119.197,-,-,19/Nov/2017:06:28:17 +0100,GET / HTTP/1.1,200,55634,http://www.ukf.sk,Go-http-client/1.1
79,194.1.157.190,-,-,19/Nov/2017:06:28:44 +0100,GET /prijimacie-konanie/moznosti-vysokoskolske...,200,9365,https://www.ukf.sk/,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
82,194.1.157.190,-,-,19/Nov/2017:06:29:03 +0100,GET /studium-na-univerzite/studijne-oddelenia ...,200,14776,https://www.ukf.sk/prijimacie-konanie/moznosti...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
...,...,...,...,...,...,...,...,...,...
1305642,93.171.214.4,-,-,19/Nov/2017:06:24:10 +0100,GET / HTTP/1.1,200,16182,-,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...
1305670,193.87.12.30,-,-,19/Nov/2017:06:25:13 +0100,GET / HTTP/1.1,200,55861,-,libwww-perl/6.04
1305671,193.87.12.30,-,-,19/Nov/2017:06:25:13 +0100,GET / HTTP/1.1,200,55846,-,libwww-perl/6.04
1305672,193.87.12.45,-,-,19/Nov/2017:06:25:13 +0100,GET / HTTP/1.1,200,56324,-,Wget/1.13.4 (linux-gnu)


###3. Vyselektujte z dátového súboru len vaše skúmané obdobie a vytvorte premennú UnixTime

 Vytvor nový stĺpec unix timestamp pomocou datetime

In [8]:
data['unixtime'] = pd.to_datetime(data['datetime'], format="%d/%b/%Y:%H:%M:%S %z")
data['unixtime'] = data['unixtime'].values.astype(np.int64) // 10 ** 9
data.head()

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent,unixtime
4,185.92.73.31,-,-,19/Nov/2017:06:28:01 +0100,GET /administrator/index.php HTTP/1.1,200,6429,https://www.ukf.sk/administrator/index.php,Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_55_...,1511069281
6,194.1.157.190,-,-,19/Nov/2017:06:28:06 +0100,GET / HTTP/1.1,200,16195,-,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,1511069286
73,139.162.119.197,-,-,19/Nov/2017:06:28:17 +0100,GET / HTTP/1.1,200,55634,http://www.ukf.sk,Go-http-client/1.1,1511069297
79,194.1.157.190,-,-,19/Nov/2017:06:28:44 +0100,GET /prijimacie-konanie/moznosti-vysokoskolske...,200,9365,https://www.ukf.sk/,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,1511069324
82,194.1.157.190,-,-,19/Nov/2017:06:29:03 +0100,GET /studium-na-univerzite/studijne-oddelenia ...,200,14776,https://www.ukf.sk/prijimacie-konanie/moznosti...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,1511069343


###4. Identifikujte používateľov na základe IP adresy a poľa User-Agent 

Usporiadaj hodnoty pomocou *ip*, a potom *useragent* a *unixtime* 

In [9]:
data = data.sort_values(by = ['clientIp', 'useragent', 'unixtime'])

Definuj stĺpec userid zo stĺpca clientIp a stĺpca useragent 

In [10]:
previousIp = ""
previousUserAgent = ""
previousUserId = 0
for i, row in data.iterrows():
  if (i == 0):
    data.at[i,'userid'] = 0
    previousIp = row['clientIp']
    previousUserAgent = row['useragent']
  else:
    if ((row['clientIp'] == previousIp) & (row['useragent'] == previousUserAgent)):
      data.at[i,'userid'] = previousUserId
    else:
      previousUserId += 1
      previousIp = row['clientIp']
      previousUserAgent = row['useragent']
    data.at[i,'userid'] = previousUserId

data.head()

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent,unixtime
857726,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,GET /wp-login.php HTTP/1.1,302,3240,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481
857734,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,GET /index.php?option=com_content&view=article...,200,40307,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481
857761,1.1.229.122,-,1,23/Nov/2017:08:01:23 +0100,GET / HTTP/1.1,200,52504,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420483
1192329,1.121.128.56,-,2,24/Nov/2017:22:45:47 +0100,GET /wp-login.php HTTP/1.1,302,3816,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559947
1192332,1.121.128.56,-,2,24/Nov/2017:22:45:48 +0100,GET /index.php?option=com_content&view=article...,200,38579,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559948


###5. Vytvorte premennú Length na základe User_ID a 60 min. STT.

In [11]:
data.reset_index(drop=True, inplace=True)
data['length'] = ''

for i, row in data.iterrows():
  if (i < len(data) - 1):
    if (data.at[i,'userid'] == data.at[i+1,'userid']):
      if ((data.at[i+1,'unixtime'] - data.at[i,'unixtime']) < 3600):
        data.at[i,'length'] = data.at[i+1,'unixtime'] - data.at[i,'unixtime']
      
data

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent,unixtime,length
0,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,GET /wp-login.php HTTP/1.1,302,3240,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,0
1,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,GET /index.php?option=com_content&view=article...,200,40307,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,2
2,1.1.229.122,-,1,23/Nov/2017:08:01:23 +0100,GET / HTTP/1.1,200,52504,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420483,
3,1.121.128.56,-,2,24/Nov/2017:22:45:47 +0100,GET /wp-login.php HTTP/1.1,302,3816,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559947,1
4,1.121.128.56,-,2,24/Nov/2017:22:45:48 +0100,GET /index.php?option=com_content&view=article...,200,38579,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559948,1
...,...,...,...,...,...,...,...,...,...,...,...
141715,99.253.144.97,-,18883,21/Nov/2017:03:50:47 +0100,GET /index.php?option=com_content&view=article...,200,39379,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511232647,1
141716,99.253.144.97,-,18883,21/Nov/2017:03:50:48 +0100,GET / HTTP/1.1,200,51112,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511232648,
141717,99.28.173.111,-,18884,25/Nov/2017:22:43:09 +0100,GET /wp-login.php HTTP/1.1,302,3778,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511646189,0
141718,99.28.173.111,-,18884,25/Nov/2017:22:43:09 +0100,GET /index.php?option=com_content&view=article...,200,37190,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511646189,1


###6. Identifikujte sedenia na základe metódy Reference Length (použite hodnotu pre podiel navigačných stránok: 40%)

In [12]:
data = data.replace('', np.nan)
data.head()

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent,unixtime,length
0,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,GET /wp-login.php HTTP/1.1,302,3240,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,0.0
1,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,GET /index.php?option=com_content&view=article...,200,40307,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,2.0
2,1.1.229.122,-,1,23/Nov/2017:08:01:23 +0100,GET / HTTP/1.1,200,52504,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420483,NaN
3,1.121.128.56,-,2,24/Nov/2017:22:45:47 +0100,GET /wp-login.php HTTP/1.1,302,3816,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559947,1.0
4,1.121.128.56,-,2,24/Nov/2017:22:45:48 +0100,GET /index.php?option=com_content&view=article...,200,38579,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559948,1.0


In [13]:
data['RLength'] = ''
count = 1
time = 0
l = 1 / data['length'].mean(skipna = True)
C = (-np.log(1-0.4)) / l

for i, row in data.iterrows():
  if (i != 0):
    if (data.at[i,'userid'] == data.at[i-1,'userid']):
      if not (math.isnan(data.at[i-1, 'length'])):
        time += data.at[i-1, 'length'].astype(int)
      else:
        count += 1
        time = 0
      if (time > C):
        count += 1
        time = 0
    else:
      count += 1
      time = 0
  data.at[i, 'RLength'] = count

data

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent,unixtime,length,RLength
0,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,GET /wp-login.php HTTP/1.1,302,3240,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,0.0,1
1,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,GET /index.php?option=com_content&view=article...,200,40307,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,2.0,1
2,1.1.229.122,-,1,23/Nov/2017:08:01:23 +0100,GET / HTTP/1.1,200,52504,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420483,NaN,1
3,1.121.128.56,-,2,24/Nov/2017:22:45:47 +0100,GET /wp-login.php HTTP/1.1,302,3816,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559947,1.0,2
4,1.121.128.56,-,2,24/Nov/2017:22:45:48 +0100,GET /index.php?option=com_content&view=article...,200,38579,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559948,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
141715,99.253.144.97,-,18883,21/Nov/2017:03:50:47 +0100,GET /index.php?option=com_content&view=article...,200,39379,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511232647,1.0,54439
141716,99.253.144.97,-,18883,21/Nov/2017:03:50:48 +0100,GET / HTTP/1.1,200,51112,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511232648,NaN,54439
141717,99.28.173.111,-,18884,25/Nov/2017:22:43:09 +0100,GET /wp-login.php HTTP/1.1,302,3778,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511646189,0.0,54440
141718,99.28.173.111,-,18884,25/Nov/2017:22:43:09 +0100,GET /index.php?option=com_content&view=article...,200,37190,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511646189,1.0,54440


###7. Doplňte chýbajúce požiadavky do identifikovaných sedení (dopĺňanie ciest)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141720 entries, 0 to 141719
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   clientIp   141720 non-null  object 
 1   user       141720 non-null  object 
 2   userid     141720 non-null  int64  
 3   datetime   141720 non-null  object 
 4   url        141720 non-null  object 
 5   req_code   141720 non-null  int64  
 6   bytes      141720 non-null  int64  
 7   referer    141720 non-null  object 
 8   useragent  141717 non-null  object 
 9   unixtime   141720 non-null  int64  
 10  length     110609 non-null  float64
 11  RLength    141720 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 13.0+ MB


Nahraď 'GET / HTTP/1.1' s 'https://www.ukf.sk/'

In [15]:
for i, row in data.iterrows():
  data.at[i,'url'] = ('https://www.ukf.sk/' + str(data.at[i,'url'])[5:-9])

data.head()

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent,unixtime,length,RLength
0,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,https://www.ukf.sk/wp-login.php,302,3240,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,0.0,1
1,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,https://www.ukf.sk/index.php?option=com_conten...,200,40307,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,2.0,1
2,1.1.229.122,-,1,23/Nov/2017:08:01:23 +0100,https://www.ukf.sk/,200,52504,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420483,NaN,1
3,1.121.128.56,-,2,24/Nov/2017:22:45:47 +0100,https://www.ukf.sk/wp-login.php,302,3816,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559947,1.0,2
4,1.121.128.56,-,2,24/Nov/2017:22:45:48 +0100,https://www.ukf.sk/index.php?option=com_conten...,200,38579,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559948,1.0,2


Vytvor triedu 'Stack'

In [16]:
class Stack:

  def __init__(self):
      self.stack = []

  def add(self, dataval):
      if dataval not in self.stack:
          self.stack.append(dataval)
          return True
      else:
          return False

  def peek(self, i):     
	    return self.stack[i]

  def clear(self):
      self.stack = []

  def length(self):
      return len(self.stack)

In [17]:
stack = Stack()
new_rows = []
stack2 = []
for i, row in data.iterrows():
  if ((i < len(data) - 1) and (i > 0)):

    if ((data.at[i, 'referer'].__contains__('https://www.ukf.sk/')) and (data.at[i+1, 'referer'].__contains__('https://www.ukf.sk/')) and (not ((data.at[i, 'url'].__contains__('image') ) == True) and (not ((data.at[i, 'url'].__contains__('iframe')== True ))))   ):
      
      if ((data.at[i,'RLength'] != data.at[i-1,'RLength']) and (data.at[i,'RLength'] == data.at[i+1,'RLength'])):
          new_rows.append(row.values)
          stack.add([data.at[i, 'url'], data.at[i+1, 'referer']])
      elif (data.at[i,'RLength'] == data.at[i+1,'RLength']):
          stack.add([data.at[i, 'url'], data.at[i+1, 'referer']])
          if ((data.at[i, 'url']) == data.at[i+1, 'referer']):
              new_rows.append(row.values)
          else:
              new_rows.append(row.values)
              a = row.copy()
              stack2 = []
              stack2.append(stack.peek(-1))
              a.url = stack2[0][1]
              a.referrer = stack2[0][0]
              a.length = 1
              new_rows.append(a)    
      else:
        new_rows.append(row.values)
    else:
      new_rows.append(row.values)
  else:
    new_rows.append(row.values)

df = pd.DataFrame(new_rows, columns = data.columns)
df

,clientIp,user,userid,datetime,url,req_code,bytes,referer,useragent,unixtime,length,RLength
0,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,https://www.ukf.sk/wp-login.php,302,3240,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,0.0,1
1,1.1.229.122,-,1,23/Nov/2017:08:01:21 +0100,https://www.ukf.sk/index.php?option=com_conten...,200,40307,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420481,2.0,1
2,1.1.229.122,-,1,23/Nov/2017:08:01:23 +0100,https://www.ukf.sk/,200,52504,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511420483,NaN,1
3,1.121.128.56,-,2,24/Nov/2017:22:45:47 +0100,https://www.ukf.sk/wp-login.php,302,3816,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559947,1.0,2
4,1.121.128.56,-,2,24/Nov/2017:22:45:48 +0100,https://www.ukf.sk/index.php?option=com_conten...,200,38579,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511559948,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
143881,99.253.144.97,-,18883,21/Nov/2017:03:50:47 +0100,https://www.ukf.sk/index.php?option=com_conten...,200,39379,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511232647,1.0,54439
143882,99.253.144.97,-,18883,21/Nov/2017:03:50:48 +0100,https://www.ukf.sk/,200,51112,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511232648,NaN,54439
143883,99.28.173.111,-,18884,25/Nov/2017:22:43:09 +0100,https://www.ukf.sk/wp-login.php,302,3778,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511646189,0.0,54440
143884,99.28.173.111,-,18884,25/Nov/2017:22:43:09 +0100,https://www.ukf.sk/index.php?option=com_conten...,200,37190,-,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...,1511646189,1.0,54440


Vytvor finálne csv

In [18]:
df.to_csv('Laca_csv_final.csv', sep=';')